In [5]:
# Import libraries
import numpy as np
import pandas as pd
from pandas import json_normalize
import ast

In [6]:
df_b = pd.read_csv('../datasets/business4.csv')

In [7]:
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,c_american_new,c_italian,c_burgers,c_mexican,c_seafood,c_fastfood,c_asian_fusion,c_bakeries,c_dietary,c_other
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,0,0,0,0,0,0,0,1,0,0
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,0,0,0,0,0,0,0,0,0,0
2,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,0,1,0,0,0,0,0,1,0,0
3,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,0,0,0,0,0,0,1,0,0,0
4,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,0,0,0,0,0,0,1,0,0,0


# 머신러닝 컬럼 늘리기

### (1) Attribute 파생 컬럼

In [8]:
# 문자열 → dict 변환 (에러 방지 포함)
df_b['attributes'] = df_b['attributes'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# 중첩 딕셔너리 확장
attr_flat = json_normalize(
    df_b.loc[df_b['attributes'].notna(), 'attributes']
)

In [9]:
# 비즈니스id 키 매칭
attr_flat['business_id'] = df_b.loc[df_b['attributes'].notna(), 'business_id'].values

# 결과 확인
attr_flat.head()

,AcceptsInsurance,AgesAllowed,Alcohol,Ambience,BYOB,BYOBCorkage,BestNights,BikeParking,BusinessAcceptsBitcoin,BusinessAcceptsCreditCards,...,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi,business_id
0,None,None,u'none',None,None,None,None,True,None,False,...,False,None,1,None,None,True,None,None,u'free',MTSW4McQd7CbVtyjqoe9mw
1,None,None,'none',"{'touristy': None, 'hipster': None, 'romantic'...",None,None,None,False,None,True,...,True,True,1,False,None,True,None,None,u'no',il_Ro8jwPlHresjw9EGmBg
2,None,None,u'none',"{'romantic': False, 'intimate': False, 'touris...",None,None,None,True,False,True,...,True,False,1,False,False,True,None,True,u'no',0bPLkL0QhhPO5kt1_EXmNQ
3,None,None,'full_bar',"{'touristy': False, 'hipster': False, 'romanti...",None,None,None,None,None,True,...,True,True,2,True,True,True,None,True,'free',MUTTqe8uqyMdBl186RmNeA
4,None,None,u'none',"{'touristy': False, 'hipster': False, 'romanti...",None,None,None,True,None,True,...,None,False,1,False,True,True,None,None,u'no',ROeacJQwBeh05Rqg7F6TCg


In [10]:
attr_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19527 entries, 0 to 19526
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AcceptsInsurance            0 non-null      object
 1   AgesAllowed                 36 non-null     object
 2   Alcohol                     17679 non-null  object
 3   Ambience                    18175 non-null  object
 4   BYOB                        2141 non-null   object
 5   BYOBCorkage                 841 non-null    object
 6   BestNights                  2535 non-null   object
 7   BikeParking                 16672 non-null  object
 8   BusinessAcceptsBitcoin      3402 non-null   object
 9   BusinessAcceptsCreditCards  18921 non-null  object
 10  BusinessParking             19041 non-null  object
 11  ByAppointmentOnly           1794 non-null   object
 12  Caters                      16245 non-null  object
 13  CoatCheck                   2263 non-null   ob

In [11]:
# -------------------------
# 1. a_full_expansion
# -------------------------

# Select relevant columns from attr_flat
cols = ["business_id", "RestaurantsDelivery", "RestaurantsTakeOut", "Caters"]
attr_subset = attr_flat[cols].copy()

# Normalize string values (e.g., 'True', 'False', None)
for c in ["RestaurantsDelivery", "RestaurantsTakeOut", "Caters"]:
    attr_subset[c] = attr_subset[c].astype(str).str.lower().replace({"none": None})

# Create 'a_full_expansion' column
attr_subset["a_full_expansion"] = (
    (attr_subset["RestaurantsDelivery"] == "true") &
    (attr_subset["RestaurantsTakeOut"] == "true") &
    (attr_subset["Caters"] == "true")
).astype(int)

# Merge with df_b on business_id
df_b = df_b.merge(
    attr_subset[["business_id", "a_full_expansion"]],
    on="business_id",
    how="left"
)

# Fill missing values with 0 (restaurants with no info)
df_b["a_full_expansion"] = df_b["a_full_expansion"].fillna(0).astype(int)

# Check results
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,c_italian,c_burgers,c_mexican,c_seafood,c_fastfood,c_asian_fusion,c_bakeries,c_dietary,c_other,a_full_expansion
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,0,0,0,0,0,0,1,0,0,0
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,0,0,0,0,0,0,0,0,0,0
2,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,1,0,0,0,0,0,1,0,0,1
3,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,0,0,0,0,0,1,0,0,0,0
4,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,0,0,0,0,0,1,0,0,0,0


In [12]:
# -------------------------
# 2. a_reservation_model
# -------------------------

# Select relevant columns from attr_flat
cols = ["business_id", "RestaurantsReservations", "RestaurantsTableService", "ByAppointmentOnly"]
attr_subset = attr_flat[cols].copy()

# Normalize string values ('True', 'False', None → lower case)
for c in ["RestaurantsReservations", "RestaurantsTableService", "ByAppointmentOnly"]:
    attr_subset[c] = attr_subset[c].astype(str).str.lower().replace({"none": None})

# Convert to boolean (True=1, False/None=0)
for c in ["RestaurantsReservations", "RestaurantsTableService", "ByAppointmentOnly"]:
    attr_subset[c] = (attr_subset[c] == "true").astype(int)

# Create 'a_reservation_model' column
attr_subset["a_reservation_model"] = (
    (attr_subset[["RestaurantsReservations", "RestaurantsTableService", "ByAppointmentOnly"]].sum(axis=1) >= 2)
).astype(int)

# Merge with df_b
df_b = df_b.merge(
    attr_subset[["business_id", "a_reservation_model"]],
    on="business_id",
    how="left"
)

# Fill missing values with 0
df_b["a_reservation_model"] = df_b["a_reservation_model"].fillna(0).astype(int)

# Check results
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,c_burgers,c_mexican,c_seafood,c_fastfood,c_asian_fusion,c_bakeries,c_dietary,c_other,a_full_expansion,a_reservation_model
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,0,0,0,0,0,1,0,0,0,0
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,0,0,0,0,0,0,0,0,0,0
2,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,0,0,0,0,0,1,0,0,1,0
3,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,0,0,0,0,1,0,0,0,0,1
4,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,0,0,0,0,1,0,0,0,0,0


In [13]:
# -------------------------
# 3. a_parking_space
# -------------------------

# Safely parse BusinessParking column
def safe_eval(x):
    try:
        if isinstance(x, str) and x.strip().startswith("{"):
            return ast.literal_eval(x)
        else:
            return {}
    except (ValueError, SyntaxError):
        return {}

attr_flat["BusinessParking_dict"] = attr_flat["BusinessParking"].apply(safe_eval)

# Extract parking features and determine availability
def has_parking(d):
    if not isinstance(d, dict):
        return 0
    # Check if any of the five options is True
    return int(any([
        d.get("street") is True,
        d.get("lot") is True,
        d.get("valet") is True,
        d.get("validated") is True,
        d.get("garage") is True
    ]))

attr_flat["a_parking_space"] = attr_flat["BusinessParking_dict"].apply(has_parking)

# Merge with df_b on business_id
df_b = df_b.merge(
    attr_flat[["business_id", "a_parking_space"]],
    on="business_id",
    how="left"
)

# Fill missing values with 0 (no info = no parking)
df_b["a_parking_space"] = df_b["a_parking_space"].fillna(0).astype(int)

# Check results
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,c_mexican,c_seafood,c_fastfood,c_asian_fusion,c_bakeries,c_dietary,c_other,a_full_expansion,a_reservation_model,a_parking_space
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,0,0,0,0,1,0,0,0,0,1
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,0,0,0,0,0,0,0,0,0,1
2,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,0,0,0,0,1,0,0,1,0,1
3,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,0,0,0,1,0,0,0,0,1,1
4,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,0,0,0,1,0,0,0,0,0,1


In [14]:
# -------------------------
# 4. a_outdoor_seating
# -------------------------

df_b = df_b.merge(
    attr_flat[['business_id', 'OutdoorSeating']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'OutdoorSeating':'a_outdoor_seating'})

In [15]:
# -------------------------
# 5. a_price_range
# -------------------------

df_b = df_b.merge(
    attr_flat[['business_id', 'RestaurantsPriceRange2']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'RestaurantsPriceRange2':'a_price_range'})

In [16]:
# -------------------------
# 6. a_good_for_group
# -------------------------

df_b = df_b.merge(
    attr_flat[['business_id', 'RestaurantsGoodForGroups']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'RestaurantsGoodForGroups':'a_good_for_group'})

In [17]:
# -------------------------
# 7. a_good_for_kids
# -------------------------

df_b = df_b.merge(
    attr_flat[['business_id', 'GoodForKids']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'GoodForKids':'a_good_for_kids'})

In [18]:
# -------------------------
# 8. a_alcohol
# -------------------------

def clean_alcohol(x):
    if pd.isna(x):
        return None
    if isinstance(x, str):
        # 앞뒤 공백 제거 + u' 접두어 제거 + 따옴표 정리
        x = x.strip().lower().replace("u'", "").replace("'", "")
        # 잘 알려진 값만 남기기
        if x in ['none', 'beer_and_wine', 'full_bar']:
            return x
        else:
            return None
    return x

attr_flat['Alcohol_clean'] = attr_flat['Alcohol'].apply(clean_alcohol)

df_b = df_b.merge(
    attr_flat[['business_id', 'Alcohol_clean']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'Alcohol_clean':'a_alcohol'})


In [19]:
# -------------------------
# 9. a_has_tv
# -------------------------

df_b = df_b.merge(
    attr_flat[['business_id', 'HasTV']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'HasTV':'a_has_tv'})

In [20]:
# -------------------------
# 10. a_noise_level
# -------------------------

def clean_noise(x):
    if pd.isna(x):
        return None
    if isinstance(x, str):
        # 소문자 통일 + 'u' 및 따옴표 제거
        x = x.strip().lower().replace("u'", "").replace("'", "")
        # 유효한 값만 남기기
        if x in ['quiet', 'average', 'loud', 'very_loud']:
            return x
        else:
            return None
    return x

attr_flat['NoiseLevel_clean'] = attr_flat['NoiseLevel'].apply(clean_noise)

attr_flat['NoiseLevel_clean'] = attr_flat['NoiseLevel'].apply(clean_noise)

df_b = df_b.merge(
    attr_flat[['business_id', 'NoiseLevel_clean']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'NoiseLevel_clean':'a_noise_level'})


In [21]:
# -------------------------
# 11. a_ambience
# -------------------------

# 딕셔너리 형태 변환 함수 정의
def parse_ambience(x):
    if pd.isna(x):
        return {}
    if isinstance(x, dict):
        return x
    try:
        return ast.literal_eval(x)
    except:
        return {}

# 대표 분위기 추출 함수 정의
def extract_main_ambience(d):
    if not isinstance(d, dict) or len(d) == 0:
        return 'none'
    
    valid_classes = ['hipster', 'romantic', 'divey', 'classy',
                     'casual', 'trendy', 'upscale', 'intimate',
                     'touristy', 'none']
    
    for k, v in d.items():
        if v is True and k in valid_classes:
            return k
    return 'none'

# 변환 적용
attr_flat['Ambience_dict'] = attr_flat['Ambience'].apply(parse_ambience)
attr_flat['Ambience_clean'] = attr_flat['Ambience_dict'].apply(extract_main_ambience)

# 원본에 병합
df_b = df_b.merge(
    attr_flat[['business_id', 'Ambience_clean']],
    on='business_id',
    how='left'
)


df_b = df_b.rename(columns={'Ambience_clean':'a_ambience'})


In [22]:
# -------------------------
# 12. a_happy_hour
# -------------------------

df_b = df_b.merge(
    attr_flat[['business_id', 'HappyHour']],
    on='business_id',
    how='left'
)

df_b = df_b.rename(columns={'HappyHour':'a_happy_hour'})

In [23]:
# 결측치로 바꿀 컬럼 목록
cols_to_clean = [
    'a_outdoor_seating', 'a_price_range', 'a_good_for_group',
    'a_good_for_kids', 'a_has_tv', 'a_noise_level', 'a_ambience', 'a_happy_hour'
]

# 각 컬럼에서 None, 'None', 'none', 'NaN', 'nan'을 np.nan으로 변환
for col in cols_to_clean:
    df_b[col] = df_b[col].replace(['None', 'none', 'NaN', 'nan'], np.nan)

### (2) 영업시간 관련 컬럼

In [24]:
# -------------------------
# 1. 요일x시간대별 영업 여부
# -------------------------


# 요일별 영업시간 컬럼 생성

df_b['hours'] = df_b['hours'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 요일 리스트 정의
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 각 요일별로 새 컬럼 생성
for day in days:
    df_b[day] = df_b['hours'].apply(lambda x: x.get(day) if isinstance(x, dict) else None)

df_b[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].head()

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
1,6:0-22:0,6:0-22:0,6:0-22:0,6:0-22:0,6:0-22:0,6:0-22:0,6:0-22:0
2,10:0-18:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,None
3,None,13:30-22:0,13:30-22:0,13:30-22:0,13:30-23:0,13:30-23:0,13:30-22:0
4,11:30-20:30,11:30-20:30,11:30-20:30,11:30-20:30,11:30-20:30,11:30-20:30,None


In [25]:

# 시간대 정의 (시작 시각, 종료 시각) in 24-hour format
time_blocks = {
    'morning': (6, 12),
    'afternoon': (12, 18),
    'evening': (18, 24),
    'night': (0, 6)
}

# 요일 리스트
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 새 컬럼 초기화
for day in days:
    for block in time_blocks:
        df_b[f"{day[:3].lower()}_{block}"] = 0

# 각 셀의 시간을 파싱하고, 블록별로 매핑
for idx, row in df_b.iterrows():
    for day in days:
        value = row[day]
        if pd.isna(value) or value == 'None':
            continue
        try:
            start_str, end_str = value.split('-')
            start = int(start_str.split(':')[0])
            end = int(end_str.split(':')[0])
        except:
            continue

        # 자정 넘는 경우 보정
        if end <= start:
            end += 24

        # 각 시간 블록과 비교
        for block, (block_start, block_end) in time_blocks.items():
            for hour in range(start, end):
                hour_mod = hour % 24
                if block_start <= hour_mod < block_end:
                    df_b.at[idx, f"{day[:3].lower()}_{block}"] = 1
                    break  # 하나라도 걸리면 break



In [26]:
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,fri_evening,fri_night,sat_morning,sat_afternoon,sat_evening,sat_night,sun_morning,sun_afternoon,sun_evening,sun_night
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,...,1,0,1,1,1,0,1,1,1,0
1,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227.0,39.637133,-86.127217,2.5,28,...,1,0,1,1,1,0,1,1,1,0
2,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771.0,27.916116,-82.760461,4.5,100,...,1,0,1,1,1,0,0,0,0,0
3,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106.0,39.953949,-75.143226,4.0,245,...,1,0,0,1,1,0,0,1,1,0
4,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147.0,39.943223,-75.162568,4.5,205,...,1,0,1,1,1,0,0,0,0,0


In [27]:
# -------------------------
# 2. 주말 영업 여부
# -------------------------

# 주말(토/일) 중 하나라도 영업하면 open_weekend = 1
df_b['open_weekend'] = (
    (df_b['sat_morning'] == 1) | (df_b['sat_afternoon'] == 1) | 
    (df_b['sat_evening'] == 1) | (df_b['sat_night'] == 1) |
    (df_b['sun_morning'] == 1) | (df_b['sun_afternoon'] == 1) |
    (df_b['sun_evening'] == 1) | (df_b['sun_night'] == 1)
).astype(int)

In [28]:
# -------------------------
# 3. 심야 영업 여부
# -------------------------

# 심야(00:00~06:00) 시간에 한 번이라도 영업하면 open_night = 1
df_b['open_night'] = (
    (df_b['mon_night'] == 1) | (df_b['tue_night'] == 1) | 
    (df_b['wed_night'] == 1) | (df_b['thu_night'] == 1) | 
    (df_b['fri_night'] == 1) | (df_b['sat_night'] == 1) | 
    (df_b['sun_night'] == 1)
).astype(int)

In [29]:
# -------------------------
# 4. 주7일 영업 여부
# -------------------------

# 주 7일 영업 여부 (매일 영업하면 open_7days = 1)
df_b['open_7days'] = (
    (
        (df_b['mon_morning'] == 1) | (df_b['mon_afternoon'] == 1) | (df_b['mon_evening'] == 1) | (df_b['mon_night'] == 1)
    ) & (
        (df_b['tue_morning'] == 1) | (df_b['tue_afternoon'] == 1) | (df_b['tue_evening'] == 1) | (df_b['tue_night'] == 1)
    ) & (
        (df_b['wed_morning'] == 1) | (df_b['wed_afternoon'] == 1) | (df_b['wed_evening'] == 1) | (df_b['wed_night'] == 1)
    ) & (
        (df_b['thu_morning'] == 1) | (df_b['thu_afternoon'] == 1) | (df_b['thu_evening'] == 1) | (df_b['thu_night'] == 1)
    ) & (
        (df_b['fri_morning'] == 1) | (df_b['fri_afternoon'] == 1) | (df_b['fri_evening'] == 1) | (df_b['fri_night'] == 1)
    ) & (
        (df_b['sat_morning'] == 1) | (df_b['sat_afternoon'] == 1) | (df_b['sat_evening'] == 1) | (df_b['sat_night'] == 1)
    ) & (
        (df_b['sun_morning'] == 1) | (df_b['sun_afternoon'] == 1) | (df_b['sun_evening'] == 1) | (df_b['sun_night'] == 1)
    )
).astype(int)

In [30]:
# -------------------------
# 5. 주간 총 영업시간
# -------------------------

# 요일 컬럼 리스트 (데이터프레임에 맞게 조정)
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def parse_hours(time_range):
    """
    '7:0-20:0' 같은 문자열을 받아서 하루 총 영업시간(시간 단위)을 반환.
    """
    if pd.isna(time_range):
        return 0
    try:
        start, end = time_range.split('-')
        start_h, start_m = map(float, start.split(':'))
        end_h, end_m = map(float, end.split(':'))
        start_time = start_h + start_m / 60
        end_time = end_h + end_m / 60
        
        # 자정 넘기는 경우 (예: 18:00~2:00)
        if end_time < start_time:
            end_time += 24
        
        return end_time - start_time
    except:
        return 0

def calc_total_weekly_hours(row):
    """
    각 요일별 영업시간을 더해서 주간 총 영업시간 계산.
    """
    total = 0
    for day in days:
        total += parse_hours(row[day])
    return total

# 주간 총 영업시간 계산
df_b['total_weekly_hours'] = df_b.apply(calc_total_weekly_hours, axis=1)


### (3) 카테고리 다양성

In [31]:
# 제외할 키워드
exclude_keywords = {"Restaurants", "Food"}

# category_variety 컬럼 생성
df_b['category_variety'] = df_b['categories'].apply(
    lambda x: len([cat.strip() for cat in str(x).split(',') if cat.strip() not in exclude_keywords])
)


### (4) 팁 리뷰 보유 여부 및 개수

In [32]:
# Load tip file
df_t = pd.read_parquet("../datasets/parquet_file/tip_filtered.parquet")

# business_id 기준으로 팁 개수 세기
tip_count = df_t.groupby('business_id')['text'].count().reset_index()
tip_count.rename(columns={'text': 'tip_count'}, inplace=True)

tip_count['has_tip'] = tip_count['tip_count'].apply(lambda x: 1 if x > 0 else 0)

df_b = df_b.merge(tip_count, on='business_id', how='left')

# NaN → 0, inf → 0으로 처리 후 int 변환
df_b['tip_count'] = df_b['tip_count'].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
df_b['has_tip'] = df_b['has_tip'].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
df_b[['business_id', 'tip_count', 'has_tip']].head()

,business_id,tip_count,has_tip
0,MTSW4McQd7CbVtyjqoe9mw,10,1
1,il_Ro8jwPlHresjw9EGmBg,5,1
2,0bPLkL0QhhPO5kt1_EXmNQ,18,1
3,MUTTqe8uqyMdBl186RmNeA,17,1
4,ROeacJQwBeh05Rqg7F6TCg,20,1


### (5) 위도/경도 기반 계산 컬럼

In [33]:
# -------------------------
# 1. 주변 식당 밀집도
# -------------------------

from sklearn.neighbors import BallTree

# 위도/경도 데이터 (radians 단위로 변환 필요)
coords = np.radians(df_b[['latitude', 'longitude']].values)

# BallTree 생성 (Haversine distance 사용)
tree = BallTree(coords, metric='haversine')

# 반경 1km = 1 / 6371 (지구 반지름 km 기준)
radius = 1 / 6371

# 각 지점에서 반경 1km 내 이웃 개수 계산 (자기 자신 포함됨)
counts = tree.query_radius(coords, r=radius, count_only=True)

# 자기 자신 제외한 식당 개수 저장
df_b['neighbor_density'] = counts - 1

In [34]:
# -------------------------
# 2. 주변 가게 중 같은 카테고리 비율
# -------------------------

# 주요 카테고리 컬럼 리스트 지정
cat_cols = [
    'c_nightlife', 'c_cafes', 'c_brunch', 'c_sandwich', 
    'c_american_trad', 'c_american_new', 'c_italian', 
    'c_burgers', 'c_mexican', 'c_seafood', 'c_fastfood',
    'c_asian_fusion', 'c_bakeries', 'c_dietary', 'c_other'
]

def calc_neighbor_similarity_bool(df_b, tree, cat_cols, radius_km=1.0):
    radius = radius_km / 6371  # km → 라디안 변환
    results = []

    # numpy 배열로 변환 (계산 속도 향상)
    cat_matrix = df_b[cat_cols].values

    for i, row in df_b.iterrows():
        # 반경 내 이웃 인덱스 검색
        ind = tree.query_radius([np.radians([row['latitude'], row['longitude']])], r=radius)[0]
        
        # 자기 자신 제외
        neighbors = df_b.iloc[ind].copy()
        neighbors = neighbors[neighbors.index != i]
        if len(neighbors) == 0:
            results.append(0)
            continue

        # 기준 매장 카테고리 (bool vector)
        base_vec = cat_matrix[i]

        # 주변 매장 중 같은 카테고리 하나라도 공유하는 비율 계산
        neighbors_vecs = cat_matrix[neighbors.index]
        shared = (neighbors_vecs & base_vec).any(axis=1)
        same_cat_ratio = shared.sum() / len(shared)
        results.append(same_cat_ratio)
    
    return results

# 적용
df_b['neighbor_similarity'] = calc_neighbor_similarity_bool(df_b, tree, cat_cols, radius_km=1.0)

In [35]:
# -------------------------
# 3. 주변 식당 평균 평점
# -------------------------

def calc_neighbor_avg_stars(df_b, tree, radius_km=1.0):
    radius = radius_km / 6371  # km → 라디안 변환
    results = []

    for i, row in df_b.iterrows():
        # 반경 내 이웃 인덱스 검색
        ind = tree.query_radius([np.radians([row['latitude'], row['longitude']])], r=radius)[0]

        # 자기 자신 제외
        neighbors = df_b.iloc[ind].copy()
        neighbors = neighbors[neighbors.index != i]
        if len(neighbors) == 0:
            results.append(np.nan)
            continue

        # 주변 가게들의 평균 별점 계산
        avg_stars = neighbors['stars'].mean()
        results.append(avg_stars)

    return results

# 적용
df_b['neighbor_avg_stars'] = calc_neighbor_avg_stars(df_b, tree, radius_km=1.0)

### (6) 리뷰 데이터 기반 파생컬럼

In [36]:
# -------------------------
# 1. 평균 리뷰 길이
# -------------------------

# Load review file
df_r = pd.read_parquet("../datasets/parquet_file/review_filtered.parquet")

# 리뷰별 텍스트 길이(단어 기준 or 글자 기준)
df_r['review_length'] = df_r['text'].fillna('').apply(lambda x: len(x.split()))  # 단어 수 기준
# 👉 글자 기준으로 하고 싶다면: len(x)


# business_id별 평균 리뷰 길이 계산
avg_review_len = (
    df_r.groupby('business_id')['review_length']
    .mean()
    .reset_index()
    .rename(columns={'review_length': 'avg_review_length'})
)

# 원본 df_b(비즈니스 테이블)와 병합
df_b = df_b.merge(avg_review_len, on='business_id', how='left')


In [37]:
# -------------------------
# 2. 평점 분산
# -------------------------

# business_id별 평점 분산 계산
df_var = (
    df_r.groupby('business_id')['stars']
    .std()
    .reset_index()
    .rename(columns={'stars': 'stars_std'})
)

# df에 병합
df_b = df_b.merge(df_var, on='business_id', how='left')

In [38]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19534 entries, 0 to 19533
Data columns (total 92 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   business_id          19534 non-null  object 
 1   name                 19534 non-null  object 
 2   address              19467 non-null  object 
 3   city                 19534 non-null  object 
 4   state                19534 non-null  object 
 5   postal_code          19532 non-null  float64
 6   latitude             19534 non-null  float64
 7   longitude            19534 non-null  float64
 8   stars                19534 non-null  float64
 9   review_count         19534 non-null  int64  
 10  is_open              19534 non-null  int64  
 11  attributes           19527 non-null  object 
 12  categories           19534 non-null  object 
 13  hours                18432 non-null  object 
 14  stability_score      19534 non-null  float64
 15  loyalty_score        19534 non-null 

### (7) vader 감성 점수

In [39]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk


In [40]:

# VADER 감성분석기 다운로드 및 초기화
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# 리뷰별 감성점수 계산
df_r['sentiment_score'] = df_r['text'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [41]:
# 식당별 평균 감성점수 계산

# business_id별 평균 감성점수
sentiment_by_business = (
    df_r.groupby('business_id')['sentiment_score']
        .mean()
        .reset_index()
        .rename(columns={'sentiment_score': 'avg_sentiment'})
)


In [ ]:
# df_b에 병합
df_b = df_b.merge(sentiment_by_business, on='business_id', how='left')


C:\Users\user\AppData\Local\Temp\ipykernel_25408\601189268.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_b['avg_sentiment'].fillna(df_b['avg_sentiment'].mean(), inplace=True)


In [48]:
df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19534 entries, 0 to 19533
Data columns (total 93 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   business_id          19534 non-null  object 
 1   name                 19534 non-null  object 
 2   address              19467 non-null  object 
 3   city                 19534 non-null  object 
 4   state                19534 non-null  object 
 5   postal_code          19532 non-null  float64
 6   latitude             19534 non-null  float64
 7   longitude            19534 non-null  float64
 8   stars                19534 non-null  float64
 9   review_count         19534 non-null  int64  
 10  is_open              19534 non-null  int64  
 11  attributes           19527 non-null  object 
 12  categories           19534 non-null  object 
 13  hours                18432 non-null  object 
 14  stability_score      19534 non-null  float64
 15  loyalty_score        19534 non-null 

# 컬럼 정리 후 csv 파일 저장

In [49]:
df_targets = df_b[['business_id', 'stability_score', 'loyalty_score', 'reliability_score']]

In [50]:
df_final = df_b.drop(columns=['stability_score', 'loyalty_score', 'reliability_score'])

In [51]:
df_final = df_final.merge(
    df_targets[['business_id', 'stability_score', 'loyalty_score', 'reliability_score']],
    on='business_id',
    how='left'
)

In [52]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19534 entries, 0 to 19533
Data columns (total 93 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   business_id          19534 non-null  object 
 1   name                 19534 non-null  object 
 2   address              19467 non-null  object 
 3   city                 19534 non-null  object 
 4   state                19534 non-null  object 
 5   postal_code          19532 non-null  float64
 6   latitude             19534 non-null  float64
 7   longitude            19534 non-null  float64
 8   stars                19534 non-null  float64
 9   review_count         19534 non-null  int64  
 10  is_open              19534 non-null  int64  
 11  attributes           19527 non-null  object 
 12  categories           19534 non-null  object 
 13  hours                18432 non-null  object 
 14  store_status         19534 non-null  object 
 15  c_nightlife          19534 non-null 

In [53]:
df_final.to_csv('../datasets/business5.csv', index=False)